## 국회의원 정보 스크래핑
- 국회의원이름, 국회의원id 추출
- 국회의원 상세 페이지 300번 요청을 보내 상세 정보 추출
- pandas DataFrame 저장
- 시각화(막대그래프, 히스토그램, 파이차트)
- DB에 members 테이블에 저장

In [1]:
import requests

from bs4 import BeautifulSoup
import re

url = 'https://www.assembly.go.kr/assm/memact/congressman/memCond/memCondListAjax.do?currentPage=1&rowPerPage=300'

html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

atag_list = soup.select('div.memberna_list dl dt a')
id_list = []
for idx, atag in enumerate (atag_list, 1):
    name = atag.text
    link = atag['href']
    matched = re.search(r'(\d+)',link)
    member_id = matched.group(0)
    
    id_list.append(member_id)

In [11]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin

member_list = []
for idx, id in enumerate (id_list, 1):
    detail_url = f'https://www.assembly.go.kr/assm/memPop/memPopup.do?dept_cd={id}'
    
    html = requests.get(detail_url).text
    soup = BeautifulSoup(html, 'html.parser')
    # 국회의원 한명의 상세정보를 저장하는 dictionary
    member_dict = {}
    
    # dt tag only
    dt_list = []
    for dt_tag in soup.select('.info_mna dl.pro_detail dt '):
        dt_list.append(dt_tag.text)
#     print(dt_list)
    
    # dd tag only
    dd_list = []
    for dd_tag in soup.select('.info_mna dl.pro_detail dd '):
        regex = re.compile(r'[\n\r\t]')
        result = regex.sub('', dd_tag.text).replace(' ','')
        dd_list.append(result)
#     print(dd_list)
    
    member_dict = dict(zip(dt_list, dd_list))
#     print(member_dict)
    
    for div_tag in soup.select('.info_mna .profile'):
        member_dict['이름'] = div_tag.select('h4')[0].text
        img_url = div_tag.find('img').attrs['src']
        member_dict['사진'] = urljoin(detail_url,img_url)
        member_dict['생년월일'] = div_tag.select_one('li:nth-of-type(4)').text
#         print(div_tag)
    
    member_list.append(member_dict)
#     print(member_dict)

len(member_list)

300

In [12]:
member_list

[{'정당': '미래통합당',
  '선거구': '경남창원시성산구',
  '소속위원회': '보건복지위원회',
  '당선횟수': '재선(19대,21대)',
  '사무실전화': '02-784-1751',
  '홈페이지': 'http://blog.naver.com/ggotop',
  '이메일': 'ggotop@naver.com',
  '보좌관': '강종길,김홍광',
  '비서관': '박응서,최광림',
  '비서': '김영록,안효상,이유진,홍지형,김지훈',
  '취미, 특기': '',
  '이름': '강기윤',
  '사진': 'https://www.assembly.go.kr/photo/9771230.jpg',
  '생년월일': '1960-06-04'},
 {'정당': '미래통합당',
  '선거구': '대구동구을',
  '소속위원회': '국방위원회',
  '당선횟수': '초선(21대)',
  '사무실전화': '',
  '홈페이지': '',
  '이메일': '',
  '보좌관': '박홍규,정운태',
  '비서관': '유진영',
  '비서': '박종원,박순권,김광연,김현정,송민욱',
  '취미, 특기': '',
  '이름': '강대식',
  '사진': 'https://www.assembly.go.kr/photo/9771142.jpg',
  '생년월일': '1959-11-02'},
 {'정당': '더불어민주당',
  '선거구': '경기안양시만안구',
  '소속위원회': '교육위원회',
  '당선횟수': '초선(21대)',
  '사무실전화': '02-784-2747~9',
  '홈페이지': 'https://blog.naver.com/dulipapa',
  '이메일': 'mainsail440@daum.net',
  '보좌관': '유진우',
  '비서관': '김도균,홍미하',
  '비서': '최경순,김혜원,최기섭,조나연,오세령',
  '취미, 특기': '',
  '이름': '강득구',
  '사진': 'https://www.assembly.go.kr/photo/9771174.jpg'

In [17]:
# [{}, {}, {}]
# {} -> Serises
# {}, {}, {} -> DataFrame

import pandas as pd

data_df = pd.DataFrame(columns=['이름', '사진', '생년월일', '정당', '선거구', '소속위원회', '당선횟수', '사무실전화', '홈페이지', '이메일', '보좌관', '비서관', '비서', '취미, 특기'] )
for member in member_list:
    series_obj = pd.Series(member)
    data_df = data_df.append(series_obj, ignore_index=True)
    
data_df.head()

,이름,사진,생년월일,정당,선거구,소속위원회,당선횟수,사무실전화,홈페이지,이메일,보좌관,비서관,비서,"취미, 특기"
0,강기윤,https://www.assembly.go.kr/photo/9771230.jpg,1960-06-04,미래통합당,경남창원시성산구,보건복지위원회,"재선(19대,21대)",02-784-1751,http://blog.naver.com/ggotop,ggotop@naver.com,"강종길,김홍광","박응서,최광림","김영록,안효상,이유진,홍지형,김지훈",
1,강대식,https://www.assembly.go.kr/photo/9771142.jpg,1959-11-02,미래통합당,대구동구을,국방위원회,초선(21대),,,,"박홍규,정운태",유진영,"박종원,박순권,김광연,김현정,송민욱",
2,강득구,https://www.assembly.go.kr/photo/9771174.jpg,1963-05-27,더불어민주당,경기안양시만안구,교육위원회,초선(21대),02-784-2747~9,https://blog.naver.com/dulipapa,mainsail440@daum.net,유진우,"김도균,홍미하","최경순,김혜원,최기섭,조나연,오세령",
3,강민국,https://www.assembly.go.kr/photo/9771233.jpg,1971-03-03,미래통합당,경남진주시을,정무위원회,초선(21대),,,,"강민승,박진우","신우성,오경훈","성환종,김민규,김오주,정재희,이승연",
4,강민정,https://www.assembly.go.kr/photo/9771283.jpg,1961-04-26,열린민주당,비례대표,교육위원회,초선(21대),,,,전경원,"김민혜,윤호숙","김수안,김성용,민지홍,서지혜,이태경",


In [18]:
print(data_df.columns)
print(data_df.index)
print(data_df.shape)

Index(['이름', '사진', '생년월일', '정당', '선거구', '소속위원회', '당선횟수', '사무실전화', '홈페이지',
       '이메일', '보좌관', '비서관', '비서', '취미, 특기'],
      dtype='object')
RangeIndex(start=0, stop=300, step=1)
(300, 14)


In [20]:
print(data_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   이름      300 non-null    object
 1   사진      300 non-null    object
 2   생년월일    300 non-null    object
 3   정당      300 non-null    object
 4   선거구     300 non-null    object
 5   소속위원회   300 non-null    object
 6   당선횟수    300 non-null    object
 7   사무실전화   300 non-null    object
 8   홈페이지    300 non-null    object
 9   이메일     300 non-null    object
 10  보좌관     300 non-null    object
 11  비서관     300 non-null    object
 12  비서      300 non-null    object
 13  취미, 특기  300 non-null    object
dtypes: object(14)
memory usage: 32.9+ KB
None


In [23]:
# 정당 unique
data_df['정당'].unique()

data_df['정당'].value_counts()

더불어민주당    176
미래통합당     103
무소속         7
정의당         6
열린민주당       3
국민의당        3
시대전환        1
기본소득당       1
Name: 정당, dtype: int64

In [24]:
data_df['선거구'].unique()

array(['경남창원시성산구', '대구동구을', '경기안양시만안구', '경남진주시을', '비례대표', '서울은평구을',
       '서울강서구갑', '세종특별자치시을', '충남아산시을', '서울광진구을', '경기안산시단원구갑', '서울노원구갑',
       '대구중구남구', '경북구미시갑', '울산동구', '강원강릉시', '서울용산구', '경기화성시병', '서울성북구을',
       '경기부천시갑', '인천서구갑', '울산남구을', '경기안산시단원구을', '부산북구강서구을', '경남양산시을',
       '부산해운대구을', '경기용인시을', '서울영등포구을', '경기의정부시을', '서울동작구갑', '경기성남시분당구을',
       '경북포항시남구울릉군', '대구서구', '경기부천시병', '경북경주시', '경기여주시양평군', '경기동두천시연천군',
       '전북전주시병', '서울노원구병', '전북익산시갑', '전남고흥군보성군장흥군강진군', '대구북구을', '경기수원시갑',
       '서울성북구갑', '경북구미시을', '서울영등포구갑', '경기수원시병', '서울서대문구을', '경기남양주시병',
       '대구달서구병', '서울송파구갑', '전남목포시', '전북전주시갑', '경기성남시분당구갑', '경북포항시북구',
       '경남김해시을', '충남논산시계룡시금산군', '경기김포시갑', '경기수원시무', '경기안산시상록구을',
       '경기성남시수정구', '경남산청군함양군거창군합천군', '충남보령시서천군', '경기남양주시을', '경북안동시예천군',
       '전남여수시을', '부산동래구', '경북군위군의성군청송군영덕군', '서울송파구병', '서울마포구갑',
       '충북청주시흥덕구', '대구동구갑', '인천남동구갑', '경기시흥시갑', '충남천안시갑', '경기안양시동안구갑',
       '광주광산구을', '경남김해시갑', '경기수원시정', '경남진주시갑', '충북보은군옥천군영동군괴산군', '대전서구을',
       '대전서

In [25]:
data_df['당선횟수'].unique()

array(['재선(19대,21대)', '초선(21대)', '재선(20대,21대)', '4선(18대,19대,20대,21대)',
       '4선(16대,17대,18대,21대)', '3선(19대,20대,21대)', '재선(17대,21대)',
       '4선(17대,18대,19대,21대)', '3선(15대,16대,21대)', '4선(17대,19대,20대,21대)',
       '재선(18대,21대)', '5선(17대,18대,19대,20대,21대)', '3선(18대,19대,21대)',
       '6선(16대,17대,18대,19대,20대,21대)', '5선(16대,17대,18대,19대,21대)',
       '5선(15대,16대,19대,20대,21대)', '5선(16대,17대,18대,20대,21대)',
       '3선(17대,19대,21대)', '3선(17대,18대,21대)', '3선(18대,20대,21대)',
       '5선(15대,16대,17대,18대,21대)'], dtype=object)

In [29]:
# 당선횟수2 컬럼을 생성
temp_str = data_df['당선횟수'].str
numberof_election = temp_str[:2]
data_df['당선횟수2'] = numberof_election
data_df.loc[:,['당선횟수', '당선횟수2']]

,당선횟수,당선횟수2
0,"재선(19대,21대)",재선
1,초선(21대),초선
2,초선(21대),초선
3,초선(21대),초선
4,초선(21대),초선
...,...,...
295,초선(21대),초선
296,"5선(15대,16대,17대,18대,21대)",5선
297,초선(21대),초선
298,초선(21대),초선


In [32]:
data_df['당선횟수2'].value_counts()

초선    151
재선     74
3선     42
4선     19
5선     13
6선      1
Name: 당선횟수2, dtype: int64

In [34]:
data_df.loc[data_df['당선횟수2'] == '6선']

,이름,사진,생년월일,정당,선거구,소속위원회,당선횟수,사무실전화,홈페이지,이메일,보좌관,비서관,비서,"취미, 특기",당선횟수2
92,박병석,https://www.assembly.go.kr/photo/9770090.jpg,1952-01-25,무소속,대전서구갑,,"6선(16대,17대,18대,19대,20대,21대)",02-788-2306,,pbs2100@gmail.com,"강창희,송석근","예석해,최명식","김혜리,김민숙,유민선,심용진,최지연",,6선


In [37]:
data_df['생년월일'].head()

0    1960-06-04
1    1959-11-02
2    1963-05-27
3    1971-03-03
4    1961-04-26
Name: 생년월일, dtype: object

In [46]:
# DatetimeIndex 클래스를 이용해서 생년월일 컬럼에서 년/월/일 값을 추출
# year, month, day column 추가
data_df['year'] = pd.DatetimeIndex(data_df['생년월일']).year
data_df['month'] = pd.DatetimeIndex(data_df['생년월일']).month
data_df['day'] = pd.DatetimeIndex(data_df['생년월일']).day

data_df.loc[:,['생년월일','year', 'month', 'day']].head()

# 가장 어린
data_df.loc[data_df['year'] == data_df['year'].max(),['이름', '정당']]

# 가장 늙은
data_df.loc[data_df['year'] == data_df['year'].min(),['이름', '정당']]

,이름,정당
65,김진표,더불어민주당
290,홍문표,미래통합당


In [57]:
# 나이를 계산하는 함수 정의
from datetime import date

my_today = date.today()
def calc_age(dtob):
    return my_today.year - dtob.year - ((my_today.month, my_today.day) < (dtob.month, dtob.day))

age_list = []
for idx, row in data_df.iterrows():
    age = calc_age(date(row['year'], row['month'], row['day']))
    age_list.append(age)
    
print(len(age_list), age_list[0:4])
data_df['나이'] = age_list
    

300 [60, 60, 57, 49]


In [61]:
data_df.loc[:,['생년월일','나이']].head()
data_df['나이'].min()
data_df['나이'].max()
data_df['나이'].mean()

55.21

In [66]:
# 선거구2 컬럼을 추가
temp_str = data_df['선거구'].str
data_df['선거구2'] = temp_str[0:2]

data_df.loc[:,['선거구','선거구2']].head()
data_df['선거구2'].value_counts(normalize=True) # %표기



경기    0.196667
서울    0.163333
비례    0.156667
부산    0.060000
경남    0.053333
경북    0.043333
인천    0.043333
대구    0.040000
충남    0.036667
전남    0.033333
전북    0.033333
광주    0.026667
충북    0.026667
강원    0.026667
대전    0.023333
울산    0.020000
제주    0.010000
세종    0.006667
Name: 선거구2, dtype: float64

In [62]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   이름      300 non-null    object
 1   사진      300 non-null    object
 2   생년월일    300 non-null    object
 3   정당      300 non-null    object
 4   선거구     300 non-null    object
 5   소속위원회   300 non-null    object
 6   당선횟수    300 non-null    object
 7   사무실전화   300 non-null    object
 8   홈페이지    300 non-null    object
 9   이메일     300 non-null    object
 10  보좌관     300 non-null    object
 11  비서관     300 non-null    object
 12  비서      300 non-null    object
 13  취미, 특기  300 non-null    object
 14  당선횟수2   300 non-null    object
 15  year    300 non-null    int64 
 16  month   300 non-null    int64 
 17  day     300 non-null    int64 
 18  나이      300 non-null    int64 
dtypes: int64(4), object(15)
memory usage: 44.7+ KB
